<a href="https://colab.research.google.com/github/geee05/Prophet-Library/blob/main/ETH_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTING THE DATA :

In [1]:
import pandas as pd

df = pd.read_csv("/content/Ethereum_tweets.csv")

df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,#1 Crypto Currency TRADING,NaN,"Profit thought trading, Learn , Trade & Earn💲....",2021-11-10 10:42:01+00:00,4,0,0,False,2021-11-15 07:26:39+00:00,Get upto 1000 that’s 10x with our premium spot...,NaN,Twitter for Android,False
1,Reyrey,Kuala Lumpur,Artist | NFT Broker & Collector for AnideaNFT ...,2011-03-07 07:22:57+00:00,343,650,589,False,2021-11-15 07:26:35+00:00,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...","['SupportingCreators', 'CryptoArt', 'CryptoArt...",Twitter for iPhone,False
2,akter jahan,NaN,NaN,2021-10-28 13:28:05+00:00,1,2,0,False,2021-11-15 07:26:32+00:00,Top 10 Coins by Social Engagement over the las...,"['LunarCrush', 'bitcoin', 'dogecoin']",Twitter Web App,False
3,riyan,NaN,i always paricipated your project,2021-09-16 16:59:11+00:00,11,127,99,False,2021-11-15 07:26:30+00:00,Requesting faucet funds into 0x5278942b39deD8c...,"['Rinkeby', 'Ethereum']",Twitter Web App,False
4,Michelle Eriksen,NaN,"Art Creator, NFT, Crypto",2021-08-11 06:40:08+00:00,522,1428,16,False,2021-11-15 07:26:30+00:00,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,"['NFTs', 'nftcollector', 'NFTCommunity', 'art'...",Twitter Web App,False


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_name         10000 non-null  object
 1   user_location     4961 non-null   object
 2   user_description  8628 non-null   object
 3   user_created      10000 non-null  object
 4   user_followers    10000 non-null  int64 
 5   user_friends      10000 non-null  int64 
 6   user_favourites   10000 non-null  int64 
 7   user_verified     10000 non-null  bool  
 8   date              10000 non-null  object
 9   text              10000 non-null  object
 10  hashtags          7528 non-null   object
 11  source            10000 non-null  object
 12  is_retweet        10000 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 879.0+ KB


In [3]:
df.isnull().sum()

user_name              0
user_location       5039
user_description    1372
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            2472
source                 0
is_retweet             0
dtype: int64

In [4]:
data = df[["user_name","text"]]
data.head()

,user_name,text
0,#1 Crypto Currency TRADING,Get upto 1000 that’s 10x with our premium spot...
1,Reyrey,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩..."
2,akter jahan,Top 10 Coins by Social Engagement over the las...
3,riyan,Requesting faucet funds into 0x5278942b39deD8c...
4,Michelle Eriksen,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...


## DATA PREPROCESSING:

In [5]:
import re
from gensim.parsing.preprocessing import remove_stopwords
import string
import unicodedata

def lower_case(text):
    return text.lower()

def remove_punctuation(text):
    text_list = [char for char in text if char not in string.punctuation]
    clean_text = "".join(text_list)
    return clean_text

def remove_numbers(text):
    text_list = []
    for i in text:
        if not re.search("\d", i):
            text_list.append(i)
    return "".join(text_list)

def remove_special_char(text):
    pattern =  r'[^a-zA-z0-9]'
    return re.sub(pattern, " ", text)


def remove_square_bracket(text):
    clean_text = re.sub(r'\[[0-9]*\]', ' ', text)
    return clean_text

def remove_white_spaces_char(text):
    clean_text = re.sub(r'\s+', ' ', text)

    return clean_text

def remove_unicode_char(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


def remove_user(text):
    clean_text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text)

    return clean_text

def remove_links(text):
    clean_text = re.sub(r'http\S+', '', text)
    clean_text = re.sub(r'bit.ly/\S+', '', text)
    clean_text = clean_text.strip('[link]')

    return clean_text

def remove_html(text):
    html = re.compile('<.*?>')

    return html.sub(r'',text)

def remove_email(text):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',text)

def remove_stopwords_from_text(text):
    clean_text = remove_stopwords(text)
    return clean_text

In [6]:
data["clean_text"] = data["text"].apply(lower_case)
data["clean_text"] = data["clean_text"].apply(remove_punctuation)
data["clean_text"] = data["clean_text"].apply(remove_numbers)
data["clean_text"] = data["clean_text"].apply(remove_special_char)
data["clean_text"] = data["clean_text"].apply(remove_square_bracket)
data["clean_text"] = data["clean_text"].apply(remove_white_spaces_char)
data["clean_text"] = data["clean_text"].apply(remove_unicode_char)
data["clean_text"] = data["clean_text"].apply(remove_user)
data["clean_text"] = data["clean_text"].apply(remove_links)
data["clean_text"] = data["clean_text"].apply(remove_html)
data["clean_text"] = data["clean_text"].apply(remove_email)
data["clean_text"] = data["clean_text"].apply(remove_stopwords_from_text)

<ipython-input-6-3c9c09841a40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["clean_text"] = data["text"].apply(lower_case)
<ipython-input-6-3c9c09841a40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["clean_text"] = data["clean_text"].apply(remove_punctuation)
<ipython-input-6-3c9c09841a40>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [7]:
data.head()

,user_name,text,clean_text
0,#1 Crypto Currency TRADING,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...
1,Reyrey,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...
2,akter jahan,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...
3,riyan,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...
4,Michelle Eriksen,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...


In [8]:
data = data.drop(data[["user_name"]], axis = 1)
data.head()

,text,clean_text
0,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...
1,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...
2,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...
3,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...
4,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...


## DERIVING THE POLARITY & LABELS FOR DATA:

In [9]:
from textblob import TextBlob

def finding_polarity(text):
    clean_text = TextBlob(str(text))
    pol = clean_text.sentiment.polarity
    if(pol==0):
        return "Neutral"
    elif(pol>0 and pol<=0.3):
        return "Weakly Positive"
    elif(pol>0.3 and pol<=0.6):
        return "Positive"
    elif(pol>0.6 and pol<=1):
        return "Strongly Positive"
    elif(pol>-0.3 and pol<=0):
        return "Weakly Negative"
    elif(pol>-0.6 and pol<=-0.3):
        return "Negative"
    elif(pol>-1 and pol<=-0.6):
        return "Strongly Negative"

In [10]:
data["polarity"] = data["clean_text"].apply(finding_polarity)
data.head()

,text,clean_text,polarity
0,Get upto 1000 that’s 10x with our premium spot...,upto s x premium spot trading signals accurate...,Weakly Positive
1,"Adopted a Hypocat #053, “Crazy Rich Cat” 🤑💵💵💵🤩...",adopted hypocat crazy rich cat hypocatclub meo...,Weakly Negative
2,Top 10 Coins by Social Engagement over the las...,coins social engagement hrs lunarcrush btc bit...,Weakly Positive
3,Requesting faucet funds into 0x5278942b39deD8c...,requesting faucet funds xbdedcbdadeaecefdca ri...,Neutral
4,New Listing BTS 0.008ETH !! \n#NFTs #nftcolle...,ew listing bts eth nfts nftcollector nftcommun...,Neutral


In [11]:
# categorizing the text

from textblob import TextBlob

def positive_negative(text):
  clean_text = TextBlob(str(text))
  pol = clean_text.sentiment.polarity

  if(pol==0):
    return 1
  elif(pol>0 and pol<=0.3):
    return 1
  elif(pol>0.3 and pol<=0.6):
    return 1
  elif(pol>0.6 and pol<=1):
    return 1
  elif(pol>-0.3 and pol<=0):
    return 0
  elif(pol>-0.6 and pol<=-0.3):
    return 0
  elif(pol>-1 and pol<=-0.6):
    return 0

In [12]:
data["label"] = data["clean_text"].apply(positive_negative)
data = data.drop(data[["text"]], axis = 1)
data.head()

,clean_text,polarity,label
0,upto s x premium spot trading signals accurate...,Weakly Positive,1.0
1,adopted hypocat crazy rich cat hypocatclub meo...,Weakly Negative,0.0
2,coins social engagement hrs lunarcrush btc bit...,Weakly Positive,1.0
3,requesting faucet funds xbdedcbdadeaecefdca ri...,Neutral,1.0
4,ew listing bts eth nfts nftcollector nftcommun...,Neutral,1.0


In [13]:
data['label'].value_counts()

1.0    8282
0.0    1640
Name: label, dtype: int64

## VECTORIZATION:

In [14]:
num_words = sum([len(word.split()) for word in data["clean_text"]])
num_words

176391

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
Vectorizer = TfidfVectorizer(max_features= 3500)

x = Vectorizer.fit_transform(data["clean_text"]).toarray()

y = data["label"].values.astype("int")

In [16]:
print(x.shape)
print(y.shape)

(10000, 3500)
(10000,)


## OBTAINING TRAIN-TEST DATA

In [17]:
# splitting the data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= 0.2, random_state= 505)

In [18]:
print(x_train.shape),
print(y_train.shape),
print(x_test.shape),
print(y_test.shape)

(8000, 3500)
(8000,)
(2000, 3500)
(2000,)


In [19]:
data['label'].value_counts()

1.0    8282
0.0    1640
Name: label, dtype: int64

## MODEL I : ComplementNB

In [22]:
from sklearn.naive_bayes import ComplementNB
clf_complement = ComplementNB()
clf_complement.fit(x_train, y_train)

ComplementNB()

In [23]:
print(f"Training Accuracy : {clf_complement.score(x_train,y_train) * 100 : 0.2f} %")
print(f"Test Accuracy : {clf_complement.score(x_test,y_test)* 100 : 0.2f} %")

Training Accuracy :  89.39 %
Test Accuracy :  85.85 %


## MODEL II : SUPPORT VECTOR MACHINE

In [25]:
from sklearn.svm import SVC

svc_model = SVC(class_weight='balanced')

svc_model.fit(x_test,y_test)

print(f"Training Accuracy: {svc_model.score(x_train, y_train)*100 : 0.2f} %")
print(f"Test Accuracy: {svc_model.score(x_test,y_test) * 100 : 0.2f} %")

Training Accuracy:  94.31 %
Test Accuracy:  99.55 %


In [28]:
y_pred = svc_model.predict(x_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9955
